# 神经网络配件

#1 损失函数-Loss
损失函数是影响深度学习性能最重要的因素之一。是外部世界对神经网络模型训练的直接指导。合适的损失函数能够保证深度学习模型收敛

##1.1 Softmax-用于分类问题
$\sigma(Z)_j=\frac{e^{Z_j}}{\sum_{k=1}^Ke^{Z_k}}, j=1,2,...,K$

比如特征的目标值是[1,2,3,4,1,2,3],那么不同值之间的距离比较相近,但是经过损失函数之后变成了[0.024,0.064,0.175,0.475,0.024,0.064,0.175]，这样差别就会很大，这样分类问题的预测结果更明显

##1.2 Cross entropy 交叉熵损失-用于回归问题
$L(w)=\frac{1}{N}\sum_{n=1}^NH(p_n,q_n)=-\frac{1}{N}\sum_{n=1}^N[y_nlog\hat{y_n}+(1-y_n)log(1-\hat{y_n})]$，求偏导有$\frac{\partial{L(w)}}{\partial{\omega}}=\frac{1}{n}\sum_xx_j(\sigma{(z)}-y),\frac{\partial{C}}{\partial{b}}=\frac{1}{n}\sum_x(\sigma{(z)}-y)$,说明$\omega,b$跟激活函数的导数没关系，并且误差越大，梯度就越大，那么参数调整的就越快

##1.3 二次代价函数
$L(\omega)=\frac{1}{2n}\sum_x||y(x)-a^L(x)||^2$
L表示代价函数，x表示样本，y表示实际值，a表示输出值，n表示样本总数

如果以一个样本举例，那么有$L=\frac{(y-a)^2}{2}$，其中$a=\sigma{(z)}, z=\sum{\omega_j*X_j+b}$，其中$\sigma$是激活函数。如果我们使用梯度下降法来调整权值参数的大小，那么就有$\frac{\partial{C}}{\partial{\omega}}=(a-y)\sigma'(z)x,\frac{\partial{C}}{\partial{b}}=(a-y)\sigma'(z)$，这说明$\omega,b$的梯度跟激活函数的梯度成正比，激活函数的梯度越大，$\omega,b$的大小调整的越快，训练收敛的就越快

##1.4 自定义损失函数
- 看中某一个属性
单独讲某一些预测值取出活赋予不同大小的参数
- 合并多个loss
多目标训练任务，设置合理的loss结合方式
- 神经网络融合
不同神经网络loss结合，共同对网络进行训练指导

如果输出神经元是线性的，那么二次代价函数就是一种合适的选择，如果输出神经元是S型函数，那么比较适合用交叉熵代价函数

#2 学习率 Learning rate
- 数值大：收敛速度快
- 数值小：精度高

选用合适的学习率的办法
- 固定一个
- 设置一个step不停迭代
- Adagrad
- RMSprop

#3 动量
正常$x += -learning_rate * dx$

#4 过拟合
模型的大部分参数能够参与运算，那么过拟合的程度就低


##4.1 正则化
没有加正则化，就是
$$Loss=\hat{y}-y\\
\Delta{w}=\frac{d(Loss)}{d(w)}\\
w := w - \eta\Delta{w}$$
假如正则化以后，
$$Loss'=\hat{y}-y+\lambda \bullet ||w^2||\\
\Delta{w}=\frac{d(Loss)}{d(w)} + 2\lambda \bullet w\\
w := w-\eta\Delta{w}-2\eta\lambda{w}$$
其中$2\eta\lambda{w}$叫做weight decay


##4.2 随机失活 Dropout
让神经元以超参数p的概率被激活或者被设置为0，每次随机选择一些神经元进行计算，剩下的不进行计算，这样就可以应对过拟合，因为只有大部分神经元都的参数都接近，才能每次选取不同的神经元才会有好的结果。一般最后两个layers用一下Dropout。训练是使用，测试集不使用

Pooling-对于原始数据进行区域求最大值或者均值的过程，本质就是降维

In [1]:
#一个3层神经网络的普通版随机失活实现
import numpy as np
p = 0.5 #激活神经元的概率，p值更高，那么随机失活更弱
def train_step(X):
    H1 = np.maximum(0, np.dot(W1, X) + b1)
    U1 = np.random.rand(*H1.shape) < p # U1是一个只包含0和1的矩阵，与H1乘完以后，就可以把某些神经元去掉
    H1 *= U1
    H2 = np.maximum(0, np.dot(W2, H1) + b2)
    U2 = np.random.rand(*H2.shape) < p
    H2 *= U2
    out = np.dot(W3, H2) + b3
    # 反向传播，计算梯度省略
    # 进行参数更新，省略
def predict(X):
    H1 = np.maximum(0, np.dot(W1, X) + b1) * p
    H2 = np.maximum(0, np.dot(W2, H1) + b2) * p
    out = np.dot(W3, H2) + b3

##4.3 最大范式约束 Max norm constraints
另一种形式的正则化，给每个神经元中权重向量的量级设定上限，并使用投影梯度下降来确保这一约束。在实践中，与之对应的是参数更新方式不变，然后要求神经元中的权重向量$\overrightarrow{\omega}$必须满足$||\overrightarrow{w}||_2 < c$这一条件，一般c值为3或者4.

#5 实现两个隐藏层的Softmax
![images](../images/11/24.png)
truncated_normal:生成截断正态分布,输出从截断正态分布抽取的随机数。产生的随机数服从指定均值和标准差的正态分布，那些到均值的距离超过2倍标准差的随机数将被丢弃，然后重新抽取，直到得到足够数量的随机数为止.x的范围是$[mean-2*stddev, mean+2*stddev]$，如果mean和stddev都是默认值，那么x的范围就是-2~2

stddev是$\frac{1}{\sqrt{weights行数}}$，这是因为$y1*W2$随后的结果是一个以y1的维度为行数目，W2的列数目为列数目的向量，这样做就是以weights矩阵的函数规范化标准差，让weights矩阵中的每一列都服从0均值阶段正态分布,这样不会给输入信号添加人为偏置![images](../images/11/25.png)如果W2的某一列的均值不为0，相当于人为假如了偏置